<a href="https://colab.research.google.com/github/TokyoYoshida/GoogleColab/blob/master/playground_deep_learning_keiba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')
# 必要ならば以下のようにディレクトリ移動する
 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd /content/drive/'My Drive'/'★個人専用ファイル'/'開発用・個人サービス開発'/'GoogleColab'/

/content/drive/My Drive/★個人専用ファイル/開発用・個人サービス開発/GoogleColab


In [0]:
%ls data/keiba


darby.csv
keiba_dat_res_20200528_2355.pkl
keiba_dat_res_20200528.pkl
keiba_dat_res_20200529_2306_20200531backup.pkl
keiba_dat_res_20200529_2306.pkl
keiba_dat_res_20200531_1207.pkl
keiba_dat_res_20200531_1707.pkl
result_20200528_2020年時点最新競馬データ.csv
調教師一覧.csv
調教師一覧.gsheet
馬一覧.csv
馬一覧.gsheet
騎手一覧.csv
騎手一覧.gsheet


In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
from pprint import pprint

In [0]:
# 行も列もなるべく省略しない
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [0]:
!pip install tensorflow==2.2.0

In [0]:
!pip install -q keras

In [0]:
import keras
from keras.utils import np_utils


Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,LeakyReLU
from keras.optimizers import RMSprop


In [0]:
# タイム値を秒数(float)に変換する。小数以下１桁まであり
def date_like_value_to_timestamp(df,col_name):
    tmp = df[col_name].str.extract('(.+):(.+)\.(.+)', expand=True)
    tmp.columns=["m","s","dec"]
    return tmp["m"].astype("float")*60+tmp["s"].astype("float")+tmp["dec"].astype("float")*0.1    

In [0]:
# 競馬データの作成

In [0]:
ldat = pd.read_pickle("./data/keiba/keiba_dat_res_20200531_1207.pkl")

In [0]:
# 前回の3位以内にマークを付ける
ldat["prev_high_rank"] = 0
ldat.loc[ldat["prev_rank"] <= 3, "prev_high_rank"] = 1

In [0]:
# 3位以内にマークを付ける
ldat["high_rank"] = 0
ldat.loc[ldat["rank"] <= 3, "high_rank"] = 1

In [0]:
# 前回ランクを段階化
ldat["prev_rank_level"] = round(ldat["prev_rank"]  / 4)

In [0]:
# ランクを段階化
ldat["rank_level"] = round(ldat["rank"]  / 4)

In [0]:
# 人気を段階化
ldat["prev_popularity_level"] = round(ldat["prev_popularity"] / 4)

In [0]:
# スピードを出す
ldat["prev_speed"] = ldat["prev_distance"] / ldat["prev_time_float"] 

In [0]:
# スピードを段階化
ldat["prev_speed_level"] =  round(ldat["prev_speed"]*10)*0.1

In [0]:
# 学習、テストデータの作成

In [0]:
ddat = ldat.dropna()

In [0]:
trainlen = int(len(ddat)*0.8)

In [0]:
train = ddat.head(trainlen)
test = ddat.tail(len(ddat)-trainlen)


In [0]:
train = ddat.head(trainlen)
test = ddat.tail(len(ddat)-trainlen)


In [0]:
# トレーニングデータは勝ち馬と負け馬をちょうど半々になるようにする start

In [0]:
winner = train.query("prev_high_rank == 1")

In [0]:
loser = train.query("prev_high_rank != 1").head(winner.count()[0])

In [0]:
catted = pd.concat([winner,loser])

In [0]:
# トレーニングデータは勝ち馬と負け馬をちょうど半々になるようにする end

In [0]:
# ランダムサンプリングしてトレーニング、テスト、最終確認用データを作る

In [0]:
train_sampled = catted.sample(frac=1)
test_sampled = test.sample(frac=1)
testorg_sampled = test.sample(frac=1)

In [0]:
test

,age,amount,condition,day,diff_arrival,diff_weight,direction,distance,frame_num,full_path,grade,ground,horse_num,jockey,month,name,odds,popularity,prev_age,prev_amount,prev_condition,prev_day,prev_diff_arrival,prev_diff_weight,prev_direction,prev_distance,prev_frame_num,prev_full_path,prev_grade,prev_ground,prev_horse_num,prev_jockey,prev_month,prev_name,prev_odds,prev_popularity,prev_race_index,prev_race_num,prev_rank,prev_return,prev_sex,prev_start_time,prev_time,prev_time_float,prev_trainer,prev_unknown,prev_weather,prev_weight,prev_winner,prev_year,prev_ymd,race_index,race_num,rank,return,sex,start_time,time,time_float,trainer,unknown,weather,weight,winner,year,ymd,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,prev_speed,prev_speed_level
529787,5,57.0,0,11,16,2.0,0,2000.0,7,html/201003010207.html,9,1,14,31,4,22946,69.3,10.0,5.0,57.0,2.0,6.0,2.0,8.0,1.0,2500.0,4.0,html/201007020110.html,9.0,1.0,8.0,130.0,3.0,22946.0,42.1,12.0,21685.0,10.0,11.0,0.0,1.0,14:50,2:42.5,162.5,160.0,-,1.0,410.0,False,2010.0,20100306.0,36901,7,6.0,0.0,1,13:30,2:01.3,121.3,160,-,1,412.0,False,2010,20100411,0,0,3.0,2.0,3.0,15.384615,15.4
529788,5,55.0,0,11,15,0.0,0,2000.0,8,html/201003010207.html,9,1,15,130,4,40,27.7,7.0,5.0,55.0,0.0,31.0,12.0,10.0,1.0,2000.0,7.0,html/201007010409.html,9.0,1.0,14.0,130.0,1.0,40.0,6.7,3.0,38902.0,9.0,16.0,0.0,0.0,14:15,2:03.6,123.6,197.0,-,1.0,482.0,False,2010.0,20100131.0,36901,7,5.0,0.0,0,13:30,2:01.3,121.3,24,-,1,482.0,False,2010,20100411,0,0,4.0,1.0,1.0,16.181230,16.2
529789,4,57.0,0,11,7,-8.0,0,2000.0,6,html/201003010207.html,9,1,11,10,4,13821,31.7,8.0,4.0,55.0,0.0,6.0,2.0,10.0,1.0,2000.0,7.0,html/201007010509.html,9.0,1.0,14.0,131.0,2.0,13821.0,33.1,7.0,12098.0,9.0,12.0,0.0,1.0,14:20,2:02.6,122.6,200.0,-,0.0,470.0,False,2010.0,20100206.0,36901,7,4.0,0.0,1,13:30,2:01.1,121.1,200,-,1,462.0,False,2010,20100411,0,0,3.0,1.0,2.0,16.313214,16.3
529790,5,57.0,0,11,16,2.0,0,2000.0,4,html/201003010207.html,9,1,8,25,4,16366,16.2,6.0,5.0,57.0,0.0,28.0,14.0,0.0,0.0,2200.0,4.0,html/201009020207.html,9.0,1.0,7.0,326.0,3.0,16366.0,44.5,10.0,4079.0,7.0,5.0,0.0,1.0,13:40,2:14.7,134.7,13.0,-,0.0,436.0,False,2010.0,20100328.0,36901,7,3.0,0.0,1,13:30,2:01.0,121.0,13,-,1,438.0,False,2010,20100411,0,1,1.0,1.0,2.0,16.332591,16.3
529791,5,57.0,0,11,14,-10.0,0,2000.0,6,html/201003010207.html,9,1,12,67,4,4027,8.3,3.0,5.0,57.0,0.0,11.0,12.0,10.0,0.0,2400.0,5.0,html/201006010406.html,9.0,0.0,10.0,75.0,1.0,4027.0,18.7,7.0,28424.0,6.0,12.0,0.0,1.0,12:50,2:41.4,161.4,232.0,-,1.0,544.0,False,2010.0,20100111.0,36901,7,2.0,0.0,1,13:30,2:01.0,121.0,232,-,1,534.0,False,2010,20100411,0,1,3.0,0.0,2.0,14.869888,14.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661585,3,57.0,15,31,115,0.0,1,2400.0,7,darby.csv,1,1,14,71,5,40328,73.8,13.0,3.0,57.0,1.0,19.0,16.0,-4.0,0.0,2000.0,6.0,html/202006030811.html,1.0,1.0,12.0,51.0,4.0,40328.0,22.6,7.0,5872.0,11.0,13.0,0.0,1.0,15:40,2:02.2,122.2,138.0,-,0.0,504.0,False,2020.0,20200419.0,46078,11,0.0,0.0,1,15:40,0,0.0,138,0,6,0.0,False,2020,20200531,0,1,3.0,0.0,2.0,16.366612,16.4
661586,3,57.0,15,31,115,0.0,1,2400.0,7,darby.csv,1,1,15,51,5,28070,14.2,4.0,3.0,57.0,1.0,19.0,5.0,-4.0,0.0,2000.0,3.0,html/202006030811.html,1.0,1.0,5.0,5.0,4.0,28070.0,3.6,2.0,5872.0,11.0,5.0,0.0,1.0,15:40,2:01.8,121.8,180.0,-,0.0,488.0,False,2020.0,20200419.0,46078,11,0.0,0.0,1,15:40,0,0.0,180,0,6,0.0,False,2020,20200531,0,1,1.0,0.0,0.0,16.420361,16.4
661587,3,57.0,15,31,115,0.0,1,2400.0,8,darby.csv,1,1,16,10,5,55059,132.2,17.0,3.0,56.0,0.0,9.0,2.0,2.0,0.0,2200.0,7.0,html/202008030511.html,2.0,1.0,11.0,10.0,5.0,55059.0,8.6,3.0,12852.0,11.0,2.0,0.0,1.0,15:35,2:11.7,131.7,124.0,外,1.0,490.0,False,2020.0,20200509.0,46078,11,0.0,0.0,1,15:40,0,0.0,124,0,6,0.0,False,2020,

In [0]:
testorg_sampled.query("race_index == 36900")

,age,amount,condition,day,diff_arrival,diff_weight,direction,distance,frame_num,full_path,grade,ground,horse_num,jockey,month,name,odds,popularity,prev_age,prev_amount,prev_condition,prev_day,prev_diff_arrival,prev_diff_weight,prev_direction,prev_distance,prev_frame_num,prev_full_path,prev_grade,prev_ground,prev_horse_num,prev_jockey,prev_month,prev_name,prev_odds,prev_popularity,prev_race_index,prev_race_num,prev_rank,prev_return,prev_sex,prev_start_time,prev_time,prev_time_float,prev_trainer,prev_unknown,prev_weather,prev_weight,prev_winner,prev_year,prev_ymd,race_index,race_num,rank,return,sex,start_time,time,time_float,trainer,unknown,weather,weight,winner,year,ymd,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,prev_speed,prev_speed_level


In [0]:
# xデータ、yデータに振り分ける

In [0]:
#  x = test_sampled[['frame_num', 'horse_num', 'name', 'sex', 'age', 'amount',
#        'jockey', 'odds', 'popularity', 'weight',
#        'diff_weight', 'trainer', 'race_num', 'grade', 'ground', 'direction',
#         'distance', 'weather', 'condition', 'start_time_float', 'year',
#        'month', 'day', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
#        'prev_name', 'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
#        'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
#        'prev_weight', 'prev_diff_weight', 'prev_trainer', 'prev_race_num',
#        'prev_grade', 'prev_ground', 'prev_direction',
#        'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float',
#        'prev_year', 'prev_month', 'prev_day']].dropna(0)

In [0]:
# x = sdat[["odds",'prev_rank','prev_diff_arrival']].dropna(0)

In [0]:
xcols = ['frame_num', 'horse_num', 'sex', 'age', 'amount',
       'odds', 'popularity', 'weight',
       'diff_weight', 'trainer', 'grade', 'ground', 'direction',
        'distance', 'weather', 'condition', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
       'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
       'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
       'prev_weight', 'prev_diff_weight', 'prev_trainer',
       'prev_grade', 'prev_ground', 'prev_direction',
       'prev_distance', 'prev_weather', 'prev_condition',"prev_speed_level"
       ]

In [0]:
# cols = ['prev_rank',"prev_speed_level"]
trainx = train_sampled[xcols]
testx = test_sampled[xcols]
lastx = testorg_sampled[xcols]

In [0]:
cols=["high_rank"]
y_np = np_utils.to_categorical(train_sampled[cols])
trainy = pd.DataFrame(y_np,columns=["lose","win"],index=train_sampled.index)

testy_np = np_utils.to_categorical(test_sampled[cols])
testy = pd.DataFrame(testy_np,columns=["lose","win"],index=test_sampled.index)

lasty_np = np_utils.to_categorical(testorg_sampled[cols])
lasty = pd.DataFrame(lasty_np,columns=["lose","win"],index=testorg_sampled.index)

In [0]:
testy

,lose,win
565873,1.0,0.0
564403,0.0,1.0
568296,0.0,1.0
571774,1.0,0.0
606063,1.0,0.0
...,...,...
625080,0.0,1.0
608865,0.0,1.0
568436,0.0,1.0
652223,1.0,0.0


In [0]:
test_sampled.query("prev_ymd == 20140831.0")

,age,amount,condition,day,diff_arrival,diff_weight,direction,distance,frame_num,full_path,grade,ground,horse_num,jockey,month,name,odds,popularity,prev_age,prev_amount,prev_condition,prev_day,prev_diff_arrival,prev_diff_weight,prev_direction,prev_distance,prev_frame_num,prev_full_path,prev_grade,prev_ground,prev_horse_num,prev_jockey,prev_month,prev_name,prev_odds,prev_popularity,prev_race_index,prev_race_num,prev_rank,prev_return,prev_sex,prev_start_time,prev_time,prev_time_float,prev_trainer,prev_unknown,prev_weather,prev_weight,prev_winner,prev_year,prev_ymd,race_index,race_num,rank,return,sex,start_time,time,time_float,trainer,unknown,weather,weight,winner,year,ymd,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,prev_speed,prev_speed_level
557038,4,57.0,0,28,2,-2.0,0,2000.0,5,html/201409040607.html,9,1,9,3,9,11966,6.6,3.0,4.0,57.0,0.0,31.0,16.0,2.0,0.0,1800.0,8.0,html/201410021007.html,9.0,1.0,13.0,53.0,8.0,11966.0,2.9,2.0,40423.0,7.0,5.0,0.0,1.0,13:20,1:46.9,106.9,141.0,-,0.0,490.0,False,2014.0,20140831.0,38795,7,12.0,0.0,1,13:15,2:02.3,122.3,141,-,0,488.0,False,2014,20140928,0,0,1.0,3.0,0.0,16.838167,16.8
583322,3,56.0,2,24,0,20.0,0,1800.0,8,html/201506010802.html,9,0,16,277,1,1199,34.7,10.0,2.0,54.0,0.0,31.0,1.0,-10.0,0.0,1800.0,8.0,html/201401020402.html,9.0,1.0,11.0,105.0,8.0,1199.0,134.9,10.0,85.0,2.0,11.0,0.0,1.0,10:20,1:53.3,113.3,220.0,-,0.0,452.0,False,2014.0,20140831.0,40626,2,1.0,34.7,1,10:30,1:55.0,115.0,220,-,0,472.0,True,2015,20150124,0,1,3.0,0.0,2.0,15.887026,15.9
657529,3,53.0,1,27,1,6.0,1,1800.0,4,html/201404030504.html,9,0,7,165,9,51493,37.1,9.0,3.0,53.0,0.0,31.0,7.0,2.0,1.0,1800.0,2.0,html/201404021004.html,9.0,0.0,2.0,165.0,8.0,51493.0,268.6,14.0,10496.0,4.0,5.0,0.0,1.0,11:40,1:55.4,115.4,135.0,-,0.0,436.0,False,2014.0,20140831.0,45796,4,6.0,0.0,1,11:35,1:55.9,115.9,135,-,0,442.0,False,2014,20140927,0,0,1.0,2.0,4.0,15.597920,15.6
590923,4,57.0,0,27,13,16.0,0,2000.0,6,html/201406040712.html,9,1,11,54,12,23206,98.3,15.0,4.0,57.0,0.0,31.0,19.0,0.0,0.0,2000.0,6.0,html/201401020409.html,9.0,1.0,10.0,18.0,8.0,23206.0,13.2,6.0,22736.0,9.0,14.0,0.0,1.0,14:15,2:05.9,125.9,77.0,-,0.0,498.0,False,2014.0,20140831.0,41158,12,17.0,0.0,1,16:05,2:08.7,128.7,77,-,0,514.0,False,2014,20141227,0,0,4.0,4.0,2.0,15.885624,15.9
546177,3,57.0,0,4,16,2.0,0,1800.0,7,html/201409040703.html,9,0,14,19,10,46440,10.5,5.0,3.0,54.0,0.0,31.0,7.0,-4.0,0.0,1800.0,8.0,html/201410021007.html,9.0,1.0,14.0,2.0,8.0,46440.0,99.2,10.0,40423.0,7.0,9.0,0.0,1.0,13:20,1:47.6,107.6,5.0,-,0.0,486.0,False,2014.0,20140831.0,38043,3,7.0,0.0,1,10:55,1:55.6,115.6,5,-,0,488.0,False,2014,20141004,0,0,2.0,2.0,2.0,16.728625,16.7
638995,3,53.0,0,19,2,4.0,1,1600.0,5,html/201405040506.html,9,0,5,58,10,37366,8.3,4.0,3.0,54.0,0.0,31.0,0.0,0.0,1.0,1800.0,2.0,html/201404021002.html,9.0,0.0,3.0,58.0,8.0,37366.0,3.2,1.0,19117.0,2.0,1.0,3.2,0.0,10:40,1:54.3,114.3,177.0,-,0.0,468.0,True,2014.0,20140831.0,44502,6,4.0,0.0,0,12:45,1:38.5,98.5,177,-,0,472.0,False,2014,20141019,1,0,0.0,1.0,0.0,15.748031,15.7
657526,3,54.0,1,27,2,0.0,1,1800.0,7,html/201404030504.html,9,0,13,143,9,20376,25.4,7.0,3.0,54.0,0.0,31.0,12.0,-2.0,1.0,1800.0,7.0,html/201404021002.html,9.0,0.0,12.0,32.0,8.0,20376.0,11.8,6.0,19117.0,2.0,4.0,0.0,0.0,10:40,1:55.4,115.4,257.0,-,0.0,472.0,False,2014.0,20140831.0,45796,4,9.0,0.0,0,11:35,1:56.4,116.4,257,-,0,472.0,False,2014,20140927,0,0,1.0,2.0,2.0,15.597920,15.6
575504,6,55.0,0,18,6,3.0,1,1400.0,8,html/201705010706.html,9,0,16,143,2,33357,85.8,13.0,3.0,54.0,0.0,31.0,11.0,2.0,1.0,1800.0,1.0,html/201404021002.html,9.0,0.0,1.0,143.0,8.0,33357.0,31.2,8.0,19117.0,2.0,7.0,0.0,0.0,10:40,1:55.8,115.8,165.0,-,0.0,492.0,False,2014.0,20140831.0,40080,6,14.0,0.0,0,13:00,1:28.5,88.5,251,-,1,510.0,False,2017,20170218,0,0,2.0,4.0,2.0,15.544041,15.5
540065,3,51.0,0,27,5,-4.0,1,1800.0,8,html/201404030509.html,9,0,14,24,9,10760,3.7,1.0,3.0,52.0,0.0,31.0,0.0,0.0,0.0,1700.0,3.0,html/201401020408.html,9.0,0.0,3.0,24.0,8.0,10760.0,1.3,1.0,11741.0,8.0,1.0

In [0]:
# 競馬データの作成 終わり

In [0]:
# 学習、テストデータの作成　終わり

In [0]:
#!pip uninstall -y keras

In [0]:
#!pip uninstall -y tensorflow

In [0]:
batch_size = 128
num_classes = 2
epochs = 2


In [0]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(trainx.shape[1],)))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(256, activation=None))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(128, activation=None))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(2)) # 出力層2ノード,全結合
model.add(Activation("sigmoid"))

In [0]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               19456     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])


In [0]:
history = model.fit(trainx, trainy,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(testx, testy))


Train on 204956 samples, validate on 111252 samples
Epoch 1/2
204956/204956 [==============================] - 17s 83us/step - loss: 1.3043 - accuracy: 0.7011 - val_loss: 0.4328 - val_accuracy: 0.7917
Epoch 2/2
204956/204956 [==============================] - 13s 64us/step - loss: 0.5188 - accuracy: 0.7294 - val_loss: 0.4272 - val_accuracy: 0.7883


In [0]:
testy

,lose,win
565873,1.0,0.0
564403,0.0,1.0
568296,0.0,1.0
571774,1.0,0.0
606063,1.0,0.0
...,...,...
625080,0.0,1.0
608865,0.0,1.0
568436,0.0,1.0
652223,1.0,0.0


In [0]:
# 予測
predictions = model.predict(lastx)

In [0]:
print(predictions)

[[0.85489297 0.07479271]
 [0.76395625 0.22338903]
 [0.819242   0.1981239 ]
 ...
 [0.8462653  0.12842661]
 [0.86664474 0.09612608]
 [0.9875549  0.00621459]]


In [0]:
# 評価
score = model.evaluate(lastx, lasty, verbose=1)


111252/111252 [==============================] - 4s 34us/step


In [0]:
len(lastx)

111252

In [0]:

print("Test score", score[0])
print("Test accuracy", score[1])

Test score 0.42720634409753233
Test accuracy 0.7882599830627441


In [0]:
predict = pd.DataFrame(predictions, columns=["predict_lose","predict_win"] ,index=lastx.index)

In [0]:
result =  pd.concat([ predict,testorg_sampled],axis=1)

In [0]:
predictions

array([[0.85489297, 0.07479271],
       [0.76395625, 0.22338903],
       [0.819242  , 0.1981239 ],
       ...,
       [0.8462653 , 0.12842661],
       [0.86664474, 0.09612608],
       [0.9875549 , 0.00621459]], dtype=float32)

In [0]:
result.to_pickle("./data/keiba/keiba_predict_20200604_1617.pkl")

In [0]:
testorg_sampled

,age,amount,condition,day,diff_arrival,diff_weight,direction,distance,frame_num,full_path,grade,ground,horse_num,jockey,month,name,odds,popularity,prev_age,prev_amount,prev_condition,prev_day,prev_diff_arrival,prev_diff_weight,prev_direction,prev_distance,prev_frame_num,prev_full_path,prev_grade,prev_ground,prev_horse_num,prev_jockey,prev_month,prev_name,prev_odds,prev_popularity,prev_race_index,prev_race_num,prev_rank,prev_return,prev_sex,prev_start_time,prev_time,prev_time_float,prev_trainer,prev_unknown,prev_weather,prev_weight,prev_winner,prev_year,prev_ymd,race_index,race_num,rank,return,sex,start_time,time,time_float,trainer,unknown,weather,weight,winner,year,ymd,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,prev_speed,prev_speed_level
634807,3,54.0,1,25,4,10.0,0,1200.0,1,html/200903010502.html,9,1,1,254,4,8915,34.8,10.0,3.0,51.0,0.0,15.0,7.0,-4.0,1.0,1600.0,7.0,html/200905010603.html,9.0,0.0,13.0,254.0,2.0,8915.0,9.7,4.0,34276.0,3.0,14.0,0.0,0.0,11:10,1:43.2,103.2,293.0,-,0.0,470.0,False,2009.0,20090215.0,44210,2,6.0,0.0,0,10:20,1:12.7,72.7,147,-,2,480.0,False,2009,20090425,0,0,4.0,2.0,1.0,15.503876,15.5
558388,3,54.0,1,11,0,-6.0,0,1700.0,2,html/201302030602.html,9,0,2,40,8,6151,11.9,5.0,3.0,54.0,1.0,27.0,5.0,4.0,0.0,1700.0,8.0,html/201302030106.html,9.0,0.0,9.0,67.0,7.0,6151.0,24.2,8.0,24337.0,6.0,4.0,0.0,0.0,12:35,1:46.9,106.9,217.0,-,1.0,460.0,False,2013.0,20130727.0,38889,2,1.0,11.9,0,10:20,1:47.8,107.8,217,-,1,454.0,True,2013,20130811,0,1,1.0,0.0,2.0,15.902713,15.9
559400,3,56.0,0,1,2,-8.0,1,1400.0,6,html/201907040208.html,9,0,9,130,12,8722,14.4,7.0,3.0,56.0,0.0,16.0,16.0,-2.0,0.0,1600.0,3.0,html/201908050507.html,9.0,1.0,5.0,130.0,11.0,8722.0,62.7,13.0,30194.0,7.0,13.0,0.0,1.0,13:20,1:35.0,95.0,17.0,外,0.0,494.0,False,2019.0,20191116.0,38961,8,7.0,0.0,1,13:45,1:24.6,84.6,17,-,0,486.0,False,2019,20191201,0,0,3.0,2.0,3.0,16.842105,16.8
575366,3,54.0,2,8,10,-4.0,0,1600.0,6,html/201408020310.html,9,1,8,9,2,45029,59.9,9.0,3.0,54.0,0.0,25.0,0.0,0.0,0.0,1400.0,8.0,html/201408010804.html,9.0,0.0,16.0,11.0,1.0,45029.0,15.4,6.0,17293.0,4.0,1.0,15.4,0.0,11:35,1:27.6,87.6,199.0,-,0.0,452.0,True,2014.0,20140125.0,40071,10,11.0,0.0,0,15:10,1:39.4,99.4,199,外,1,448.0,False,2014,20140208,1,0,0.0,3.0,2.0,15.981735,16.0
600832,3,56.0,1,28,7,4.0,0,1800.0,5,html/201006030202.html,9,0,9,29,3,61110,11.0,5.0,3.0,56.0,0.0,21.0,6.0,0.0,1.0,1600.0,1.0,html/201005010805.html,9.0,1.0,2.0,29.0,2.0,61110.0,5.8,2.0,24370.0,5.0,8.0,0.0,1.0,12:20,1:40.1,100.1,29.0,-,0.0,532.0,False,2010.0,20100221.0,41853,2,13.0,0.0,1,10:15,1:59.6,119.6,29,-,1,536.0,False,2010,20100328,0,0,2.0,3.0,0.0,15.984016,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534140,3,54.0,1,25,0,2.0,0,1200.0,5,html/201209020206.html,9,1,9,107,3,23576,8.0,4.0,3.0,54.0,0.0,11.0,2.0,-6.0,0.0,1400.0,7.0,html/201209010611.html,2.0,1.0,14.0,10.0,3.0,23576.0,120.5,15.0,7863.0,11.0,8.0,0.0,0.0,15:35,1:23.7,83.7,168.0,-,1.0,434.0,False,2012.0,20120311.0,37204,6,1.0,8.0,0,12:30,1:11.1,71.1,168,-,0,436.0,True,2012,20120325,0,1,2.0,0.0,4.0,16.726404,16.7
608330,5,57.0,0,2,7,-2.0,0,2000.0,6,html/200701010809.html,2,1,11,60,9,1391,7.4,4.0,5.0,57.5,0.0,22.0,7.0,8.0,0.0,2000.0,2.0,html/200702020409.html,3.0,1.0,2.0,3.0,7.0,1391.0,3.5,1.0,31722.0,9.0,4.0,0.0,1.0,15:25,2:03.1,123.1,179.0,-,0.0,526.0,False,2007.0,20070722.0,42369,9,9.0,0.0,1,15:25,2:00.8,120.8,179,-,0,524.0,False,2007,20070902,0,0,1.0,2.0,0.0,16.246954,16.2
610600,3,56.0,0,3,18,14.0,0,1800.0,7,html/201006030306.html,9,0,9,18,4,50243,19.5,6.0,2.0,55.0,0.0,21.0,0.0,0.0,1.0,1400.0,5.0,html/200905050505.html,9.0,0.0,9.0,67.0,11.0,50243.0,4.8,3.0,26061.0,5.0,1.0,4.8,1.0,12:10,1:28.1,88.1,33.0,-,0.0,476.0,True,2009.0,20091121.0,42526,6,10.0,0.0,1,12:50,1:59.1,119.

In [0]:
result["final_return"] = result["return"]
result.loc[result["return"] == 0, "final_return"] = -1


In [0]:
# 購入する馬の決定

In [0]:
result["luck"] = result["predict_win"]/ (1/result["odds"])

In [0]:
result.query("predict_win > 0.1")[["predict_win","odds","luck"]]

,predict_win,odds,luck
558388,0.223389,11.9,2.658329
559400,0.198124,14.4,2.852984
600832,0.196395,11.0,2.160343
540988,0.165803,14.1,2.337826
613702,0.271359,9.4,2.550773
...,...,...,...
621200,0.145073,29.3,4.250627
594259,0.182014,10.0,1.820138
534140,0.292937,8.0,2.343494
608330,0.267390,7.4,1.978687


In [0]:
# 単純に勝率が高い馬を買うパターン
# predict_win = result.query("predict_win > 0.8 and luck >  5")

In [0]:
# そのレースで一番勝率の高い馬を買うパターン
predict_win = result.loc[result.groupby("race_index")["predict_win"].idxmax()]

In [0]:
len(predict_win)

8223

In [0]:
predict_win[["odds","predict_win"]].head(5)

,odds,predict_win
529792,3.7,0.328676
529803,4.7,0.345052
529821,4.4,0.452572
529832,1.5,0.404769
529843,2.4,0.372201


In [0]:

return_amount = predict_win.sum()["return"]
highrank_amount = predict_win.query("high_rank == 1").count()["name"]
count_amount = predict_win.count()["name"]

In [0]:
# 回収率
return_amount / count_amount

0.7795816611942114

In [0]:
# 的中率
highrank_amount / count_amount

0.6053751672139122

In [0]:
# 回収額
return_amount - count_amount

-1812.5

In [0]:
result.query("prev_ymd == 20140831.0 and prev_race_num == 11.0")

,predict_lose,predict_win,age,amount,condition,day,diff_arrival,diff_weight,direction,distance,frame_num,full_path,grade,ground,horse_num,jockey,month,name,odds,popularity,prev_age,prev_amount,prev_condition,prev_day,prev_diff_arrival,prev_diff_weight,prev_direction,prev_distance,prev_frame_num,prev_full_path,prev_grade,prev_ground,prev_horse_num,prev_jockey,prev_month,prev_name,prev_odds,prev_popularity,prev_race_index,prev_race_num,prev_rank,prev_return,prev_sex,prev_start_time,prev_time,prev_time_float,prev_trainer,prev_unknown,prev_weather,prev_weight,prev_winner,prev_year,prev_ymd,race_index,race_num,rank,return,sex,start_time,time,time_float,trainer,unknown,weather,weight,winner,year,ymd,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,prev_speed,prev_speed_level,final_return,luck
583515,0.906542,0.042428,7,58.0,0,5,2,-4.0,0,1600.0,5,html/201409040811.html,9,1,6,2,10,15948,34.2,10.0,7.0,58.0,0.0,31.0,17.0,-6.0,0.0,1800.0,2.0,html/201410021011.html,9.0,1.0,2.0,6.0,8.0,15948.0,8.9,6.0,41534.0,11.0,12.0,0.0,1.0,15:35,1:46.9,106.9,46.0,-,1.0,522.0,False,2014.0,20140831.0,40640,11,6.0,0.0,1,15:30,1:34.2,94.2,46,外,1,518.0,False,2014,20141005,0,0,3.0,2.0,2.0,16.838167,16.8,-1.0,1.451051
583509,0.940965,0.052359,7,58.0,0,5,14,4.0,0,1600.0,6,html/201409040811.html,9,1,8,11,10,19694,49.2,11.0,7.0,58.0,0.0,31.0,2.0,4.0,0.0,1800.0,7.0,html/201410021011.html,9.0,1.0,10.0,11.0,8.0,19694.0,34.8,10.0,41534.0,11.0,3.0,0.0,1.0,15:35,1:45.6,105.6,203.0,-,1.0,522.0,False,2014.0,20140831.0,40640,11,12.0,0.0,1,15:30,1:34.7,94.7,203,外,1,526.0,False,2014,20141005,1,0,1.0,3.0,2.0,17.045455,17.0,-1.0,2.576043
583511,0.891516,0.062768,7,56.0,0,5,15,24.0,0,1600.0,7,html/201409040811.html,9,1,11,25,10,1447,26.3,9.0,7.0,56.0,0.0,31.0,1.0,-10.0,0.0,1800.0,1.0,html/201410021011.html,9.0,1.0,1.0,13.0,8.0,1447.0,6.4,3.0,41534.0,11.0,8.0,0.0,2.0,15:35,1:46.3,106.3,125.0,-,1.0,524.0,False,2014.0,20140831.0,40640,11,10.0,0.0,2,15:30,1:34.4,94.4,125,外,1,548.0,False,2014,20141005,0,0,2.0,2.0,1.0,16.933208,16.9,-1.0,1.650800
575399,0.964653,0.059579,5,56.0,0,2,2,-4.0,0,1800.0,4,html/201408040911.html,9,1,6,66,11,2220,106.9,13.0,5.0,56.0,0.0,31.0,5.0,-4.0,0.0,1800.0,5.0,html/201410021011.html,9.0,1.0,6.0,14.0,8.0,2220.0,14.4,8.0,41534.0,11.0,9.0,0.0,2.0,15:35,1:46.5,106.5,225.0,-,1.0,482.0,False,2014.0,20140831.0,40073,11,7.0,0.0,2,15:30,1:47.7,107.7,225,外,1,478.0,False,2014,20141102,0,0,2.0,2.0,2.0,16.901408,16.9,-1.0,6.369032
659957,0.818969,0.123851,2,54.0,0,20,2,-6.0,0,1800.0,3,html/201409040309.html,9,1,3,12,9,25938,17.8,6.0,2.0,54.0,0.0,31.0,2.0,4.0,1.0,1600.0,4.0,html/201404021011.html,3.0,1.0,8.0,12.0,8.0,25938.0,24.7,7.0,36906.0,11.0,10.0,0.0,1.0,15:45,1:34.6,94.6,3.0,外,0.0,480.0,False,2014.0,20140831.0,45965,9,4.0,0.0,1,14:25,1:48.7,108.7,3,外,1,474.0,False,2014,20140920,0,0,2.0,1.0,2.0,16.913319,16.9,-1.0,2.204542
642433,0.530104,0.407135,2,55.0,0,24,2,4.0,1,1800.0,2,html/201405050711.html,3,1,2,72,11,35500,2.4,1.0,2.0,54.0,0.0,31.0,11.0,6.0,1.0,1600.0,4.0,html/201404021011.html,3.0,1.0,7.0,72.0,8.0,35500.0,3.2,1.0,36906.0,11.0,2.0,0.0,1.0,15:45,1:33.4,93.4,82.0,外,0.0,494.0,False,2014.0,20140831.0,44744,11,2.0,0.0,1,15:25,1:47.9,107.9,82,-,0,498.0,False,2014,20141124,1,1,0.0,0.0,0.0,17.130621,17.1,-1.0,0.977125
598647,0.613635,0.355814,2,55.0,0,2,0,0.0,0,1800.0,3,html/201403030610.html,9,1,3,110,11,32454,2.2,1.0,2.0,54.0,0.0,31.0,5.0,0.0,1.0,1600.0,5.0,html/201404021011.html,3.0,1.0,10.0,110.0,8.0,32454.0,64.9,12.0,36906.0,11.0,4.0,0.0,1.0,15:45,1:33.8,93.8,135.0,外,0.0,478.0,False,2014.0,20140831.0,41695,10,1.0,2.2,1,14:40,1:50.4,110.4,135,-,0,478.0,True,2014,20141102,0,1,1.0,0.0,3.0,17.057569,17.1,2.2,0.782791
553210,0.956938,0.040096,2,55.0,0,9,12,4.0,1,2000.0,7,html/201405050209.html,9,1,7,58,11,44140,62.7,9.0,2.0,54.0,0.0,31.0,17.0,0.0,1.0,1600.0,7.0,html/201404021011.html,3.0,1.0,13.0,58.0,8.0,44140.0,51.7,9.0,36906.0,11.0,15.0,0.0,1.0,15:45,1:36.0,96.0,27.0,外,0.0,426.0,False,2014.0,20140831.0,38533,9,8.0,0.0,1,14:25,2:02

In [0]:
predict_win

,predict_lose,predict_win,age,amount,condition,day,diff_arrival,diff_weight,direction,distance,frame_num,full_path,grade,ground,horse_num,jockey,month,name,odds,popularity,prev_age,prev_amount,prev_condition,prev_day,prev_diff_arrival,prev_diff_weight,prev_direction,prev_distance,prev_frame_num,prev_full_path,prev_grade,prev_ground,prev_horse_num,prev_jockey,prev_month,prev_name,prev_odds,prev_popularity,prev_race_index,prev_race_num,prev_rank,prev_return,prev_sex,prev_start_time,prev_time,prev_time_float,prev_trainer,prev_unknown,prev_weather,prev_weight,prev_winner,prev_year,prev_ymd,race_index,race_num,rank,return,sex,start_time,time,time_float,trainer,unknown,weather,weight,winner,year,ymd,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,prev_speed,prev_speed_level,final_return,luck
529792,0.587415,0.328676,5,57.0,0,11,0,0.0,0,2000.0,4,html/201003010207.html,9,1,7,2,4,12560,3.7,2.0,3.0,55.0,0.0,2.0,16.0,0.0,0.0,2400.0,8.0,html/200808040809.html,9.0,1.0,12.0,45.0,11.0,12560.0,2.3,1.0,5969.0,9.0,3.0,0.0,1.0,14:15,2:25.2,145.2,157.0,外,0.0,490.0,False,2008.0,20081102.0,36901,7,1.0,3.7,1,13:30,2:00.8,120.8,157,-,1,490.0,True,2010,20100411,1,1,1.0,0.0,0.0,16.528926,16.5,3.7,1.216100
529803,0.522563,0.345052,3,54.0,1,3,2,-4.0,1,1400.0,2,html/201805010309.html,9,1,2,32,2,52232,4.7,3.0,3.0,54.0,0.0,13.0,7.0,14.0,0.0,1600.0,4.0,html/201806010409.html,9.0,1.0,4.0,32.0,1.0,52232.0,5.5,2.0,40406.0,9.0,3.0,0.0,0.0,14:25,1:36.3,96.3,34.0,外,0.0,512.0,False,2018.0,20180113.0,36902,9,5.0,0.0,0,14:35,1:22.7,82.7,34,-,1,508.0,False,2018,20180203,1,0,1.0,1.0,0.0,16.614746,16.6,-1.0,1.621745
529821,0.577380,0.452572,2,54.0,0,2,17,-4.0,0,1400.0,7,html/201908050111.html,3,1,12,102,11,49876,4.4,1.0,2.0,54.0,0.0,29.0,2.0,8.0,0.0,1600.0,5.0,html/201906040909.html,9.0,1.0,5.0,102.0,9.0,49876.0,4.2,2.0,23934.0,9.0,2.0,0.0,0.0,14:25,1:34.3,94.3,180.0,外,1.0,432.0,False,2019.0,20190929.0,36903,11,2.0,0.0,0,15:45,1:20.9,80.9,180,外,0,428.0,False,2019,20191102,1,1,0.0,0.0,0.0,16.967126,17.0,-1.0,1.991317
529832,0.542498,0.404769,3,54.0,0,27,15,2.0,0,1800.0,6,html/201606020107.html,9,1,8,102,2,678,1.5,1.0,3.0,54.0,0.0,23.0,7.0,-4.0,0.0,1800.0,5.0,html/201606010709.html,9.0,1.0,7.0,67.0,1.0,678.0,5.0,3.0,44779.0,9.0,2.0,0.0,0.0,14:25,1:49.6,109.6,80.0,-,1.0,490.0,False,2016.0,20160123.0,36904,7,3.0,0.0,0,13:30,1:48.9,108.9,80,-,0,492.0,False,2016,20160227,1,1,0.0,1.0,1.0,16.423358,16.4,-1.0,0.607154
529843,0.565349,0.372201,4,57.0,0,27,2,0.0,1,2000.0,8,html/201305040907.html,9,1,12,41,10,4222,2.4,1.0,4.0,57.0,0.0,14.0,14.0,6.0,1.0,1800.0,8.0,html/201305040507.html,9.0,1.0,13.0,41.0,10.0,4222.0,4.4,2.0,36381.0,7.0,2.0,0.0,1.0,13:30,1:48.3,108.3,32.0,-,0.0,494.0,False,2013.0,20131014.0,36905,7,4.0,0.0,1,13:20,2:01.2,121.2,32,-,0,494.0,False,2013,20131027,1,0,0.0,1.0,0.0,16.620499,16.6,-1.0,0.893283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661524,0.560056,0.417219,3,54.0,1,10,3,-8.0,0,1700.0,2,html/201603020404.html,9,0,3,41,7,3595,3.6,1.0,3.0,54.0,0.0,11.0,5.0,4.0,1.0,1600.0,8.0,html/201605030301.html,9.0,0.0,15.0,41.0,6.0,3595.0,3.9,1.0,12716.0,1.0,2.0,0.0,0.0,10:10,1:38.7,98.7,272.0,-,0.0,436.0,False,2016.0,20160611.0,46074,4,9.0,0.0,0,11:40,1:49.8,109.8,272,-,0,428.0,False,2016,20160710,1,0,0.0,2.0,0.0,16.210740,16.2,-1.0,1.501988
661548,0.582808,0.465650,3,55.0,0,13,2,-2.0,1,1600.0,3,html/200704010605.html,9,1,5,46,5,24408,3.5,1.0,3.0,56.0,0.0,11.0,15.0,0.0,0.0,2000.0,5.0,html/200710010804.html,9.0,1.0,5.0,189.0,2.0,24408.0,2.8,1.0,39386.0,4.0,4.0,0.0,1.0,11:20,2:05.5,125.5,55.0,-,0.0,490.0,False,2007.0,20070211.0,46075,5,8.0,0.0,1,11:45,1:38.0,98.0,55,外,0,488.0,False,2007,20070513,0,0,1.0,2.0,0.0,15.936255,15.9,-1.0,1.629774
661559,0.619260,0.395718,5,55.0,0,4,7,0.0

In [0]:
# ダービー予想 start

In [0]:
darby = ddat.query("full_path == 'darby.csv'")[xcols]

In [0]:
darby

,frame_num,horse_num,sex,age,amount,odds,popularity,weight,diff_weight,trainer,grade,ground,direction,distance,weather,condition,prev_rank,prev_frame_num,prev_horse_num,prev_sex,prev_age,prev_amount,prev_jockey,prev_time_float,prev_diff_arrival,prev_odds,prev_popularity,prev_weight,prev_diff_weight,prev_trainer,prev_grade,prev_ground,prev_direction,prev_distance,prev_weather,prev_condition,prev_speed_level
661572,1,1,1,3,57.0,38.9,7.0,0.0,0.0,15,1,1,1,2400.0,6,15,13.0,8.0,17.0,1.0,3.0,57.0,51.0,93.7,5.0,5.1,3.0,482.0,-6.0,15.0,1.0,1.0,1.0,1600.0,0.0,0.0,17.1
661573,1,2,1,3,57.0,61.2,12.0,0.0,0.0,10,1,1,1,2400.0,6,15,2.0,2.0,2.0,1.0,3.0,56.0,270.0,108.0,15.0,2.1,1.0,456.0,-6.0,10.0,3.0,1.0,0.0,1800.0,1.0,1.0,16.7
661574,2,3,1,3,57.0,12.4,3.0,0.0,0.0,239,1,1,1,2400.0,6,15,2.0,7.0,8.0,1.0,3.0,56.0,5.0,123.2,1.0,2.5,1.0,494.0,2.0,239.0,2.0,1.0,0.0,2000.0,1.0,2.0,16.2
661575,2,4,1,3,57.0,98.5,15.0,0.0,0.0,271,1,1,1,2400.0,6,15,11.0,1.0,2.0,1.0,3.0,57.0,10.0,122.1,16.0,62.9,9.0,482.0,-4.0,271.0,1.0,1.0,0.0,2000.0,0.0,1.0,16.4
661576,3,5,1,3,57.0,1.5,1.0,0.0,0.0,15,1,1,1,2400.0,6,15,1.0,1.0,1.0,1.0,3.0,57.0,3.0,120.7,0.0,2.7,1.0,462.0,0.0,15.0,1.0,1.0,0.0,2000.0,0.0,1.0,16.6
661577,3,6,1,3,57.0,54.7,9.0,0.0,0.0,10,1,1,1,2400.0,6,15,8.0,8.0,17.0,1.0,3.0,57.0,17.0,121.9,11.0,13.0,4.0,486.0,-4.0,10.0,1.0,1.0,0.0,2000.0,0.0,1.0,16.4
661578,4,7,1,3,57.0,133.8,18.0,0.0,0.0,200,1,1,1,2400.0,6,15,9.0,5.0,9.0,1.0,3.0,57.0,139.0,122.1,5.0,99.5,12.0,432.0,2.0,200.0,1.0,1.0,0.0,2000.0,0.0,1.0,16.4
661579,4,8,1,3,57.0,56.6,11.0,0.0,0.0,200,1,1,1,2400.0,6,15,1.0,1.0,1.0,1.0,3.0,56.0,39.0,119.8,0.0,3.9,2.0,464.0,0.0,200.0,9.0,1.0,1.0,2000.0,1.0,0.0,16.7
661580,5,9,1,3,57.0,30.0,6.0,0.0,0.0,257,1,1,1,2400.0,6,15,6.0,7.0,13.0,1.0,3.0,57.0,138.0,121.8,2.0,18.5,6.0,516.0,-10.0,257.0,1.0,1.0,0.0,2000.0,0.0,1.0,16.4
661581,5,10,1,3,57.0,91.9,14.0,0.0,0.0,12,1,1,1,2400.0,6,15,7.0,2.0,3.0,1.0,3.0,57.0,66.0,121.9,7.0,100.9,13.0,460.0,6.0,12.0,1.0,1.0,0.0,2000.0,0.0,1.0,16.4


In [0]:
darby_predictions = model.predict(darby)

In [0]:
darby_predict = pd.DataFrame(darby_predictions, columns=["predict_lose","predict_win"] ,index=darby.index)

In [0]:
winner = darby_predict.sort_values("predict_win",ascending=False).head(3)

In [0]:
ldat.loc[winner.index]["name"]

661576    40318
661589    40321
661584    40325
Name: name, dtype: int64

In [0]:
# ダービー予想 end